In [1]:
!pip install -U huggingface_hub
!pip install -U datasets
!pip install transformers[torch]
!pip install evaluate
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.2 MB/s eta 0:00:00


## **Clasificacion con Transformers**

In [2]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv('canciones.csv')

# Renombrar la columna 'tag' a 'labels'
df = df.rename(columns={'tag': 'labels'})

# Crear un mapeo de etiquetas a índices
label2id = {label: idx for idx, label in enumerate(df['labels'].unique())}
id2label = {idx: label for label, idx in label2id.items()}

# Mapear etiquetas a índices numéricos
df['labels'] = df['labels'].map(label2id)

# Seleccionar columnas relevantes
df = df[['lyrics', 'labels', 'split']]

# Dividir en conjuntos de entrenamiento, validación y prueba
train_data = df[df['split'] == 'train']
val_data = df[df['split'] == 'validation']
test_data = df[df['split'] == 'test']

# Convertir a objetos Dataset de Huggingface
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)
test_dataset = Dataset.from_pandas(test_data)


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Seleccionar modelo preentrenado
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Cargar modelo con el número de etiquetas según géneros únicos
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
def tokenize_function(examples):
    tokens = tokenizer(examples['lyrics'], padding="max_length", truncation=True)
    tokens['labels'] = examples['labels']  # Asegurar que las etiquetas se copien correctamente como números
    return tokens

# Tokenizar los datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Formatear los datasets para entrenamiento
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/21000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

In [5]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score

from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Cálculo de las métricas
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    precision, recall, _, support = precision_recall_fscore_support(labels, preds, average=None)

    # Retorna todas las métricas en el diccionario
    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision.tolist(),
        "recall": recall.tolist(),
        "support": support.tolist()
    }



training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,  # Incluye el tokenizer
    compute_metrics=compute_metrics,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-5-626570c58982>:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [6]:
print(train_dataset[0]["labels"])  # Verifica que las etiquetas sean enteros


tensor(0)


CLAVE: d6f16458f7d148cb6dbc35d20b8c5521c64a98e5

In [7]:
# Entrenar el modelo
trainer.train()

# Evaluar en el conjunto de prueba
results = trainer.evaluate(test_dataset)
print(results)


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Support
1,0.825600,0.775606,0.668000,0.666414,"[0.4692387904066736, 0.8960603520536463, 0.5542521994134897, 0.7428861788617886]","[0.4147465437788018, 0.9082412914188616, 0.675, 0.6538461538461539]","[1085, 1177, 1120, 1118]"
2,0.700300,0.769726,0.670667,0.672635,"[0.43845534995977475, 0.9008403361344538, 0.6214285714285714, 0.7286108555657773]","[0.5023041474654378, 0.9107901444350043, 0.54375, 0.7084078711985689]","[1085, 1177, 1120, 1118]"
3,0.534600,0.844629,0.679333,0.677918,"[0.4759660697455231, 0.8964076858813701, 0.614115490375802, 0.7028670721112077]","[0.46543778801843316, 0.9116397621070518, 0.5982142857142857, 0.723613595706619]","[1085, 1177, 1120, 1118]"


Trainer is attempting to log a value of "[0.4692387904066736, 0.8960603520536463, 0.5542521994134897, 0.7428861788617886]" of type <class 'list'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.4147465437788018, 0.9082412914188616, 0.675, 0.6538461538461539]" of type <class 'list'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[1085, 1177, 1120, 1118]" of type <class 'list'> for key "eval/support" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.43845534995977475, 0.9008403361344538, 0.6214285714285714, 0.7286108555657773]" of type <class 'list'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scala

Trainer is attempting to log a value of "[0.48148148148148145, 0.8818565400843882, 0.5597426470588235, 0.7163543441226575]" of type <class 'list'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.44551845342706503, 0.9024179620034543, 0.5665116279069767, 0.7449069973427812]" of type <class 'list'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[1138, 1158, 1075, 1129]" of type <class 'list'> for key "eval/support" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.8552080988883972, 'eval_accuracy': 0.6671111111111111, 'eval_f1': 0.6643408310178515, 'eval_precision': [0.48148148148148145, 0.8818565400843882, 0.5597426470588235, 0.7163543441226575], 'eval_recall': [0.44551845342706503, 0.9024179620034543, 0.5665116279069767, 0.7449069973427812], 'eval_support': [1138, 1158, 1075, 1129], 'eval_runtime': 67.3802, 'eval_samples_per_second': 66.785, 'eval_steps_per_second': 8.356, 'epoch': 3.0}


## **Generacion de texto con GPT-2**

In [ ]:
# Sección 4: Generación de texto con GPT-2
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import pandas as pd

# Configurar el modelo y tokenizador
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "gpt2-xl"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer.pad_token = tokenizer.eos_token


In [ ]:
# Texto de entrada
input_txt = "Genre: pop Lyrics: A romantic song about love and heartbreak."
input_tokens = tokenizer(input_txt, return_tensors="pt", padding=True, truncation=True)
input_ids = input_tokens["input_ids"].to(device)
attention_mask = input_tokens["attention_mask"].to(device)


In [ ]:
# Función para generación de texto
def generate_text(input_ids, attention_mask, method="greedy", **kwargs):
    if method == "greedy":
        output = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_new_tokens=50,
            do_sample=False,
            no_repeat_ngram_size=2
        )
    elif method == "beam":
        output = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_new_tokens=50,
            num_beams=5,
            no_repeat_ngram_size=2,
        )
    elif method == "sampling":
        output = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_new_tokens=50,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.9,
            no_repeat_ngram_size=2
        )
    else:
        raise ValueError("Método desconocido: elija 'greedy', 'beam' o 'sampling'")

    return tokenizer.decode(output[0], skip_special_tokens=True)

# Generaciones con diferentes métodos
print("\nGreedy Search:")
print(generate_text(input_ids, attention_mask, method="greedy"))

print("\nBeam Search:")
print(generate_text(input_ids, attention_mask, method="beam"))

print("\nSampling:")
print(generate_text(input_ids, attention_mask, method="sampling"))

In [ ]:
# Configuración del tokenizador y modelo
tokenizer.pad_token = tokenizer.eos_token
input_tokens = tokenizer(input_txt, return_tensors="pt", padding=True, truncation=True)
input_ids = input_tokens["input_ids"].to(device)
attention_mask = input_tokens["attention_mask"].to(device)

# Función para calcular probabilidad logarítmica
def log_probs_from_logits(logits, labels):
    logp = torch.nn.functional.log_softmax(logits, dim=-1)
    return torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)

def sequence_logprob(model, labels, input_len=0):
    with torch.no_grad():
        output = model(labels)
        log_probs = log_probs_from_logits(output.logits[:, :-1, :], labels[:, 1:])
        seq_log_prob = torch.sum(log_probs[:, input_len:])
        return seq_log_prob.cpu().numpy()

# Generación con Greedy Search
output_greedy = model.generate(
    input_ids,
    attention_mask=attention_mask,
    max_new_tokens=50,
    do_sample=False
)

# Calcular Log-prob para evaluación
logp_greedy = sequence_logprob(model, output_greedy, input_len=len(input_ids[0]))
print(f"\nLog-prob (Greedy): {logp_greedy:.2f}")
